# Play with the data from the original Open TDE Catalog by James Guillochan

In [2]:
%load_ext autoreload
%autoreload 2

import os, glob, warnings
import numpy as np
import pandas as pd
import json
from otter import Otter, Transient
from astroquery.simbad import Simbad
import astropy.units as u

In [3]:
bandwavelengths = {
    "u"      : 354.,
    "g"      : 475.,
    "r"      : 622.,
    "i"      : 763.,
    "z"      : 905.,
    "u'"     : 354.,
    "g'"     : 475.,
    "r'"     : 622.,
    "i'"     : 763.,
    "z'"     : 905.,
    "u_SDSS" : 354.3,
    "g_SDSS" : 477.0,
    "r_SDSS" : 623.1,
    "i_SDSS" : 762.5,
    "z_SDSS" : 913.4,
    "U"      : 365.,
    "B"      : 445.,
    "V"      : 551.,
    "R"      : 658.,
    "I"      : 806.,
    "Y"      : 1020.,
    "J"      : 1220.,
    "H"      : 1630.,
    "K"      : 2190.,
    "M2"     : 260.,
    "W1"     : 224.6,
    "W2"     : 192.8,
    "w"      : 622.,
}

bandreps = {
    'Ks': ['K_s'],
    'M2': ['uvm2', 'UVM2', 'UVm2', 'Um2', 'm2', 'um2'],
    'W1': ['uvw1', 'UVW1', 'UVw1', 'Uw1', 'w1', 'uw1'],
    'W2': ['uvw2', 'UVW2', 'UVw2', 'Uw2', 'w2', 'uw2']
}

xraycodes = [
    "0.3 - 10",
    "0.5 - 8"
]

# from https://imagine.gsfc.nasa.gov/science/toolbox/spectrum_chart.html
bandOptions = {'radio':{'min':1e-3, 'max':1e-100}, 
               'x-ray':{'min':1e-11, 'max':1e-8}, 
               'optical':{'min':4e-7, 'max':7e-7}, 
               'infared':{'min':7e-7, 'max':1e-3}, 
               'uv':{'min':1e-8, 'max':4e-7}
              }

In [4]:
keymap = {'name': 'name/default_name',
         'sources': 'reference_alias',
         'alias': 'name/alias',
         'claimedtype': 'class',
         'discoverdate': 'epoch/date_discovery',
         'redshift': 'distance/redshift',
         'lumdist': 'distance/luminosity_distance',
          'ra': 'coordinate/equitorial',
          'dec': 'coordinate/equitorial',
          
         }

In [5]:
datapath = os.path.relpath('../data/tde-1980-2025')

for file in glob.glob(os.path.join(datapath, '*.json')):
    with open(file, 'r') as f:
        j = json.load(f)[0]
        
    print(json.dumps(j, indent=4))
    
    df = pd.DataFrame(j['photometry'])
    print(df)
    
    break

{
    "schema": "https://github.com/astrocatalogs/tidaldisruptions/blob/f405406/SCHEMA.md",
    "name": "SDSSJ1323",
    "sources": [
        {
            "name": "2017ApJ...838..149A",
            "bibcode": "2017ApJ...838..149A",
            "reference": "Auchettl, Guillochon, & Ramirez-Ruiz (2017)",
            "alias": "1"
        },
        {
            "name": "2016A&A...594A..13P",
            "bibcode": "2016A&A...594A..13P",
            "reference": "Planck Collaboration et al. (2016)",
            "alias": "2"
        },
        {
            "name": "2007A&A...462L..49E",
            "bibcode": "2007A&A...462L..49E",
            "reference": "Esquej et al. (2007)",
            "alias": "3"
        },
        {
            "name": "The Open TDE Catalog",
            "bibcode": "2017ApJ...835...64G",
            "reference": "Guillochon et al. (2017)",
            "secondary": true,
            "url": "https://tde.space",
            "alias": "4"
        }
    ],
    "alias"

# Everything Below this point is old code

In [3]:
cat = TDECatalog(username='admin@tide', password='insecure', debug=False)

In [4]:
def lum(F,z,wave):
    """
    Calculates the luminosity from the flux and redshift
    From Collin Christy
    
    F [array[float]] : flux in mJy
    nu [float] : freq in GHz
    z [float] : redshift
    """

    if isinstance(F, u.quantity.Quantity):
        F = F.value
    
    if isinstance(F, str):
        F = float(F)
    
    c = 299792.458 # km/s
    H0 = 70 #km/s/Mpc
    d = z*c/H0 # Mpc
    D = d*3.086e24 # converts to cm
    F_cgs = F*1e-3*1e-23 # 10^-3 converts mJy -> Jy, 10^-23 coverts 1Jy to ergs s^-1 cm^2 Hz^-1
    nu_effective = c*1e12/wave # converts to Hz
    L = F_cgs*4*np.pi*D**2*nu_effective
    return L

def getLuminosity(photo, wave, z=None):
    '''
    Short function to get the luminosity of a photometry point
    '''
    if 'luminosity' in photo:
        return photo
    elif 'flux' in photo:
        photo['luminosity'] = lum(float(photo['flux']), z, wave)
        return photo
    elif 'magnitude' in photo:
        appMag = (float(photo['magnitude'])*u.STmag).to(u.erg/u.s/u.cm**2/u.AA)
        photo['luminosity'] = lum(appMag, z, wave)
    elif 'fluxdensity' in photo:
        photo['luminosity'] = lum(photo['fluxdensity'], z, wave)
    

In [5]:
datapath = os.path.relpath('../data/tde-1980-2025')
tdes = []
notadded = {}
notaddedreason = []
for file in glob.glob(os.path.join(datapath, '*.json')):
    with open(file, 'r') as f:
        j = json.load(f)[0]
    # handle missing ra and decs
    if 'ra' not in j:
        try:
            j['ra'] = j['hostra']
        except KeyError:
            # try querying Simbad
            res = Simbad.query_object(j['name'])
            j['ra'] = [{'value': res['RA'][0], 'bibcode': res['COO_BIBCODE'][0]}]
            warnings.warn(f"Need to add source by hand for RA for {j['name']}")
    if 'dec' not in j:
        try:
            j['dec'] = j['hostdec']
        except KeyError:
            # try querying Simbad
            res = Simbad.query_object(j['name'])
            j['dec'] = [{'value': res['DEC'][0], 'bibcode': res['COO_BIBCODE'][0]}]
            warnings.warn(f"Need to add source by hand for DEC for {j['name']}")
    
    # handle difference in naming convention for redshift
    if 'redshift' in j:
        j['z'] = j['redshift']
        del j['redshift']
    # handle the photometry
    photoDict = {}
    if 'photometry' in j:
        for photo in j['photometry']:
            if 'energy' in photo:
                if 'x-ray' in photoDict:
                    photoDict['x-ray'].append(photo)
                else:
                    photoDict['x-ray'] = [photo]
            elif 'band' in photo:
                try:
                    wave = bandwavelengths[photo['band']]
                except KeyError:
                    for band in bandreps:
                        if photo['band'] in bandreps[band]:
                            wave = bandwavelengths[band]
                            break

                found = False
                for key in bandOptions:
                    if wave*10**-9 > bandOptions[key]['min'] and wave*10**-9 < bandOptions[key]['max']:
                        if key in photoDict:
                            photoDict[key].append(photo)
                        else:
                            photoDict[key] = [photo]
                        found = True
                        break
                if not found:
                    raise Exception('This photometry does not match any!')

            else:
                if j['name'] not in notadded:
                    notadded[j['name']] = []
                notadded[j['name']].append(photo)
                notaddedreason.append('This photometry does not have "energy" or "band"')
                continue
                
                
            # convert to luminosity
            if 'z' in j and 'luminosity' not in photo:
                getLuminosity(photo, float(j['z'][0]['value']), wave) 
            elif 'luminosity' in photo:
                pass
            else:
                if j['name'] not in notadded:
                    notadded[j['name']] = []
                notadded[j['name']].append(photo)
                notaddedreason.append('No luminosity or redshift found!')
                continue
                
            # make sure it has flux
            if 'magnitude' not in photo and 'flux' in photo:
                j['magnitude'] = 22.5 - 2.5*np.log10(float(photo['flux']))
            
            elif 'magnitude' not in photo:
                if j['name'] not in notadded:
                    notadded[j['name']] = []
                notadded[j['name']].append(photo)
                notaddedreason.append('This photometry does not have "magnitude" or "flux"')
                continue

        j['photometry'] = photoDict
        
    # make this json file into a TDE
    for source in j['sources']:
        if 'reference' in source:
            source['name'] = source['reference']
            del source['reference']
        if 'bibcode' not in source:
            try:
                source['bibcode'] = source['url']
            except KeyError:
                source['bibcode'] = ''
    
    if 'spectra' in j:
        # this needs to be fixed later
        j['spectra'] = {'spec':j['spectra']}
        
    if 'discoverydate' in j:
        j['discovery_date'] = j['discoverydate']
        del j['discoverydate']
    
    try:
        t = TDE(j)
    except Exception as e:
        if 'name' not in j: j['name'] = 'no-name-found'
        if j['name'] not in notadded:
            notadded[j['name']] = []
        notadded[j['name']].append(j)
        notaddedreason.append(e)
    
    tdes.append(t)

/tmp/ipykernel_1049232/2505054958.py:16: UserWarning:

Need to add source by hand for RA for NGC247

/tmp/ipykernel_1049232/2505054958.py:24: UserWarning:

Need to add source by hand for DEC for NGC247



In [6]:
cat.upload(tdes, test=False)

{'radio': [{'dt_days': 8, 'dt_years': 0.0219178082, 'nu_GHz': 6.1, 'F_mJy': 0.033, 'luminosity': 1.035098148e+37, 'time': 8, 'upperlimit': True, 'source': '1', 'uuid': '84445a1c-8242-477f-85f4-71e4d8014ea7'}, {'dt_days': 23, 'dt_years': 0.0630136986, 'nu_GHz': 6.1, 'F_mJy': 0.036, 'luminosity': 1.129197979e+37, 'time': 23, 'upperlimit': True, 'source': '1', 'uuid': '873cc351-407d-4e52-8637-ae780517db22'}, {'dt_days': 90, 'dt_years': 0.2465753425, 'nu_GHz': 6.1, 'F_mJy': 0.06, 'luminosity': 1.881996632e+37, 'time': 90, 'upperlimit': True, 'source': '1', 'uuid': '565f0de3-2c86-46a1-afc9-0c9f2e9f74c5'}, {'dt_days': 182, 'dt_years': 0.498630137, 'nu_GHz': 4.8, 'F_mJy': 1.114, 'luminosity': 2.749566228e+38, 'time': 182, 'upperlimit': False, 'source': '1', 'uuid': 'af3800e9-91ea-4286-b048-339dee7b27f7'}, {'dt_days': 190, 'dt_years': 0.5205479452, 'nu_GHz': 4.8, 'F_mJy': 0.899, 'luminosity': 2.218904882e+38, 'time': 190, 'upperlimit': False, 'source': '1', 'uuid': 'e5b8ff57-31bb-4dba-8c99-9de

In [7]:
len(tdes)

98

In [8]:
len(notadded)

6

In [9]:
notadded.keys()

dict_keys(['MAXI J1807+132', 'no-name-found', 'XMMSL1 J0740-85', 'AT2018zr', 'ASASSN-14li', 'Dougie'])

In [10]:
notaddedreason

['No luminosity or redshift found!',
 'No luminosity or redshift found!',
 'No luminosity or redshift found!',
 'No luminosity or redshift found!',
 'No luminosity or redshift found!',
 'No luminosity or redshift found!',
 Exception('time, luminosity, and source are required for every photometry point'),
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'This photometry does not have "energy" or "band"',
 'Thi